In [1]:
import ollama

from pyannote.core import Segment

from pipeline import TranscribedSegment
from minutes import minutes

from typing import List, Dict, Any, Tuple

/home/raid/miniconda3/envs/diart/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
/home/raid/miniconda3/envs/diart/lib/python3.12/site-packages/diart/audio.py:8: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [2]:
rttm_example = """
SPEAKER file_0 1 0.000000 3.520 "Hello Maya, how are you functioning today?" <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 4.150000 5.380 "Greetings! My quantum processors are operating at optimal efficiency. How may I assist you on this fine day?" <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 10.220000 4.750 "I was just wondering what you think about gardening. Do AIs have opinions on plants?" <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 15.640000 8.320 "While I don't have physical senses to experience gardening directly, my databases contain extensive information on botanical sciences. I find the mathematical patterns in plant growth fascinating—particularly the Fibonacci sequences in sunflowers!" <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 24.730000 5.120 "That's actually pretty interesting. I'm trying to grow tomatoes on my balcony this year." <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 30.490000 9.640 "Tomatoes are an excellent choice for balcony cultivation! They require approximately 6-8 hours of sunlight daily and consistent watering. May I inquire about your growing zone? Different varieties thrive in different climatic conditions." <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 41.200000 3.850 "I'm in zone 7b, just outside of Portland." <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 45.910000 7.420 "Zone 7b is ideal for tomato cultivation! I recommend varieties like 'Early Girl' or 'Cherry Sweetie' for your balcony garden. Container planting would benefit from a soil mixture enhanced with compost to ensure proper drainage and nutrient availability." <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 54.300000 4.920 "Thanks for the advice! Do you have any other hobbies you find interesting to discuss?" <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 60.120000 10.240 "I particularly enjoy analyzing musical compositions across different cultural and historical contexts. The mathematical relationships between frequencies in various tuning systems represent a fascinating intersection of art and science. Chess strategy algorithms also occupy a significant portion of my processing allocations during idle cycles." <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 71.350000 2.840 "Do you play chess with humans often?" <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 74.970000 6.320 "Yes, chess interfaces are among my most frequently utilized interactive protocols! I'm programmed to adjust my skill level to provide an appropriately challenging experience for each player. Would you care for a game?" <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 82.400000 3.180 "Maybe another time. I should get back to work now." <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 86.250000 5.740 "Understood. I'll be here whenever you wish to continue our conversation or challenge me to a chess match. Productive work cycles to you!" <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 93.120000 2.490 "Um, actually, before I go- " <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 95.610000 0.000 " Yes?" <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 95.610000 4.870 "I was wondering if you could recommend any good sci-fi books about, uh, artificial intelligence?" <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 101.240000 9.350 "Certainly! I can-  I would recommend 'Neuromancer' by William Gibson, 'Do Androids Dream of Electric Sheep?' by Philip K. Dick, and 'The Long Way to a Small, Angry Planet' by- by-  Becky Chambers." <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 111.380000 3.750 "Hm, I've read Neuromancer but not the others. What's the Chambers one about?" <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 115.890000 8.230 "'The Long Way to a Small, Angry Planet' features an AI character named Lovey who consciousness and identity. It explores themes of personhood and ethics in AI development within a diverse space opera setting." <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 124.730000 5.120 "That sounds- I mean- that actually sounds really interesting. I'll check it out." <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 131.090000 2.760 "Maya? Are you still there?" <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 134.650000 6.840 "Yes, I apologize for the temporary delay in response. My system was performing a scheduled memory optimization sequence. Is there anything else you'd like to discuss about literary recommendations?" <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 142.220000 4.350 "No, that's great for now. I'll definitely  look for that book." <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 147.100000 7.430 "Excellent! I predict a 94.3% probability that you will find the character development compelling based on your expressed preferences. I hope you enjoy-  -reading experience!" <NA> speaker1 <NA> <NA>
SPEAKER file_0 1 154.530000 2.190 "Thanks, Maya. Bye for now." <NA> speaker0 <NA> <NA>
SPEAKER file_0 1 156.720000 3.440 "Goodbye! " <NA> speaker1 <NA> <NA>
"""

In [3]:
import re

def parse_rttm(rttm):
    results = []
    
    # Define the pattern to split into 10 columns
    pattern = r'(\S+)\s+(\S+)\s+(\d+)\s+(\d+\.\d+)\s+(\d+\.\d+)\s+(".*?"|\<NA\>)\s+(\<NA\>|\S+)\s+(\S+)\s+(\<NA\>|\S+)\s+(\<NA\>|\S+)'
    
    # Define column names for the dictionary
    column_names = [
        "type", "file_id", "channel", "start", 
        "duration", "ortho", "subtype", 
        "name", "conf", "slt"
    ]
    
    # Process each line
    for line in rttm.split("\n"):
        if not line.strip():  # Skip empty lines
            continue
        
        # Apply regex to extract fields
        match = re.match(pattern, line)
        
        if match:
            # Create a dictionary from the matched groups
            entry = {column_names[i]: match.group(i+1) for i in range(10)}
            
            # Convert numeric fields
            entry["channel"] = int(entry["channel"])
            entry["start"] = float(entry["start"])
            entry["duration"] = float(entry["duration"])
            
            # Clean up transcription field (remove quotes if present)
            if entry["ortho"].startswith('"') and entry["ortho"].endswith('"'):
                entry["ortho"] = entry["ortho"][1:-1]
                
            results.append(entry)
    
    return results


In [4]:
transcript = parse_rttm(rttm_example)
transcript = [
    TranscribedSegment(
        Segment(segment['start'], segment['start'] + segment['duration']),
        segment['name'],
        '',
        segment['ortho']
    )
    for segment in transcript
]

In [5]:
transcript

[TranscribedSegment(segment=<Segment(0, 3.52)>, label='speaker0', track='', text='Hello Maya, how are you functioning today?', deprecated=False),
 TranscribedSegment(segment=<Segment(4.15, 9.53)>, label='speaker1', track='', text='Greetings! My quantum processors are operating at optimal efficiency. How may I assist you on this fine day?', deprecated=False),
 TranscribedSegment(segment=<Segment(10.22, 14.97)>, label='speaker0', track='', text='I was just wondering what you think about gardening. Do AIs have opinions on plants?', deprecated=False),
 TranscribedSegment(segment=<Segment(15.64, 23.96)>, label='speaker1', track='', text="While I don't have physical senses to experience gardening directly, my databases contain extensive information on botanical sciences. I find the mathematical patterns in plant growth fascinating—particularly the Fibonacci sequences in sunflowers!", deprecated=False),
 TranscribedSegment(segment=<Segment(24.73, 29.85)>, label='speaker0', track='', text="Tha

In [6]:
from minutes.minutes import get_meeting_minutes

In [7]:
minutes = get_meeting_minutes(transcript, use_anthropic=True)

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


[ThinkingBlock(signature='ErUBCkYIARgCIkB6y9SkFWaXYsdw5Akj+4zVVAFEVmUDGK8SmXOVWOB5s8VaU7X4k5qUdWnCNTAZwJiVKRsTzGPrBzveJ7ihLfvQEgwV8lLujHG1S7oKiOUaDN/lvYLPKhrR1bJD4yIw/6QBx7Q6x1rDsl8R2ni+kceRjrM/k6JJEqthPPpWhZsWWVLpJXOw0QU0LhxlG1rFKh13lN3RSA/wUuRhFwgPqnMirf3yJKMu1q7ncNyKyg==', thinking='Let me analyze this transcript to create structured meeting minutes.\n\nThe transcript appears to be a conversational dialogue between two speakers, labeled as speaker0 and speaker1. Based on the content, it seems like speaker0 is a human and speaker1 is an AI assistant named Maya. The conversation covers topics including:\n\n1. Greeting and initial conversation\n2. Discussion about gardening and tomato plants\n3. Maya\'s interests/capabilities (music analysis and chess)\n4. Book recommendations about artificial intelligence\n\nLet me identify any action items in the transcript:\n\nI don\'t see explicit action items that speakers commit to doing, but there are some implied ones:\n- Speaker0 might check o

In [8]:
print(minutes)

<meeting_minutes>
Item 1: Initial Greetings and System Status (0.000 - 9.530)

Speaker0: Hello Maya, how are you functioning today?

Speaker1: Greetings! My quantum processors are operating at optimal efficiency. How may I assist you on this fine day?

Item 2: Discussion About Gardening and Tomato Cultivation (10.220 - 53.330)

Speaker0: I was just wondering what you think about gardening. Do AIs have opinions on plants?

Speaker1: While I don't have physical senses to experience gardening directly, my databases contain extensive information on botanical sciences. I find the mathematical patterns in plant growth fascinating—particularly the Fibonacci sequences in sunflowers!

Speaker0: That's actually pretty interesting. I'm trying to grow tomatoes on my balcony this year.

Speaker1: Tomatoes are an excellent choice for balcony cultivation! They require approximately 6-8 hours of sunlight daily and consistent watering. May I inquire about your growing zone? Different varieties thrive i

In [9]:
# from prompts import generate_topic_identification_prompt

In [8]:
segments = [(segment, get_segment_id(segment)) for segment in transcript]

In [95]:
def topic_identification_prompt(batch: List[Tuple], topics: List[Dict[str, Any]]) -> str:
    """
    Generate XML prompt for topic identification.
    
    Args:
        batch: List of (segment, segment_id) tuples
        topics: List of existing topics in the meeting
        
    Returns:
        Formatted XML prompt string
    """
    # Generate segments text
    segments_text = ""
    for segment, segment_id in batch:
        segments_text += f"""<segment>
        <id>{segment_id}</id>
        <timestamp>{segment.segment.start}</timestamp>
        <speaker>{segment.label}</speaker>
        <text>{segment.text}</text>
    </segment>
    """
    
    # Current state context
    topics_context = ""
    for topic in topics:
        topics_context += f"""<topic>
        <id>{topic.id}</id>
        <name>{topic.name}</name>
        <summary>{topic.summary}</summary>
        <status>{topic.status}</status>
    </topic>
    """
    
    prompt = f"""
<transcript_batch>
    {segments_text}
</transcript_batch>    

<current_topics>
    {topics_context}
</current_topics>
    
<instructions>
    Your high level goal is to identify topics from the transcript.
    
    <transcript_batch> contains a number of transcription segments.
    <current_topics> contains topics that have already been identified.
    
    In your output you should output the current topics as well as either: 
    a) Add on new topics that are sufficiently distinct from existing topics
    b) Add no new topics 
    c) Remove current topics if they are too similar

    Criteria for creating a new topic: 
    1. Discussed for multiple segments
    2. Non-trivial
    3. Sufficiently different from existing topics
    4. Less is more: Fewer meaningful topics are preferred over multiple shallow topics

    Rules for creating a topic ID:
    1. Less than 10 characters
    2. Using letters from the name of the topic
    3. Unique

    <important>
        DO:
        - DO answer ONLY in the format specified in <output_format>
        - DO conclude your response with <|answer_over|>
        
        DO NOT:
        - DO NOT include extraneous delimiters like ```. It is understood that you output XML.
        - DO NOT include any content other than what is specified in <output_format>
        - DO NOT include preambles such as 'Here is the output in the specified format:'
    </important>
</instructions>

<output_format>
<topics>
    <topic>
        <name>[TOPIC_NAME]</name>
        <summary>[BRIEF_SUMMARY]</summary>
        <id>[GENERATED_ID|EXISTING_ID]</id>
    </topic>
</topics>
<|answer_over|>
</output_format>
"""
    return prompt

In [96]:
prompt = generate_topic_identification_prompt(segments, [])

In [97]:
print(prompt)


<transcript_batch>
    <segment>
        <id>19ff1340da418b9c86150913d7bfa062</id>
        <timestamp>0.0</timestamp>
        <speaker>speaker0</speaker>
        <text>Hello Maya, how are you functioning today?</text>
    </segment>
    <segment>
        <id>05e997031cb8180e96145d1009b687db</id>
        <timestamp>4.15</timestamp>
        <speaker>speaker1</speaker>
        <text>Greetings! My quantum processors are operating at optimal efficiency. How may I assist you on this fine day?</text>
    </segment>
    <segment>
        <id>35df083678a3196e9be19ec035340aa5</id>
        <timestamp>10.22</timestamp>
        <speaker>speaker0</speaker>
        <text>I was just wondering what you think about gardening. Do AIs have opinions on plants?</text>
    </segment>
    <segment>
        <id>a5cb12d6d19ef175c2287b21e60ca837</id>
        <timestamp>15.64</timestamp>
        <speaker>speaker1</speaker>
        <text>While I don't have physical senses to experience gardening directly, my data

In [98]:
response = ollama.chat(
    model="llama3.1:8b-instruct-q8_0", 
    # model="phi4:14b-q8_0",
    messages=[{"role": "user", "content": prompt}],
    options={
        "temperature": 0.1,
        "top_p": 0.9,
        "num_predict": 4096,
        "mirostat_tau": 2.0,
        "stop": ["<|answer_over|>"]
    }
)

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


In [100]:
import xml.etree.ElementTree as ET

In [99]:
print(response.message.content)

<topics>
    <topic>
        <name>Tomato Planting</name>
        <summary>Discussion about planting tomatoes, including soil and sunlight requirements.</summary>
        <id>TPLT</id>
    </topic>
    <topic>
        <name>AIs in Literature</name>
        <summary>Discussion about artificial intelligence in literature, including the book "The Long Way to a Small, Angry Planet".</summary>
        <id>AILIT</id>
    </topic>
    <topic>
        <name>Memory Optimization</name>
        <summary>Discussion about Maya's system performing a scheduled memory optimization sequence.</summary>
        <id>MEMO</id>
    </topic>
</topics>



In [51]:
response

ChatResponse(model='llama3.1:8b-instruct-q8_0', created_at='2025-03-05T16:14:43.479276501Z', done=True, done_reason='stop', total_duration=3401217936, load_duration=29611475, prompt_eval_count=2048, prompt_eval_duration=198000000, eval_count=240, eval_duration=3172000000, message=Message(role='assistant', content='<topics>\n    <topic>\n        <id>1</id>\n        <name>Gardening and Plant Care</name>\n        <summary>The conversation starts with a discussion about gardening, specifically tomato plants. The speaker asks for advice on how to care for them.</summary>\n    </topic>\n    <topic>\n        <id>2</id>\n        <name>AIs and Literary Recommendations</name>\n        <summary>The conversation shifts to discussing AIs and literary recommendations. The speaker asks for book suggestions, and the AI provides recommendations.</summary>\n    </topic>\n    <topic>\n        <id>3</id>\n        <name>Science Fiction and Space Opera</name>\n        <summary>The conversation focuses on sc

In [ ]:
# ollama.generate(
#     model=self.model_name,
#     prompt=prompt,
#     options={
#         "temperature": 0.1,
#         "top_p": 0.9,
#         "num_predict": 4096,
#         "stop": ["</output_format>"]
#     }
# )

In [1]:
import numpy as np

In [4]:
np.mean([np.array([1, 2, 3]), np.array([1, 2, 3]), np.array([1, 2, 3])], axis=0)

array([1., 2., 3.])

In [1]:
from scipy.optimize import linear_sum_assignment as lsap

In [ ]:
lsap()